# Defining an ODE system

There are 2 ways to encapsulate an ODE system.

We first demonstrate how to encapsulate the relevant equations in a class which will eventually be used to solve them.

One useful feature of PyGom is the ability to define the system in a variety of intuitive ways.

In compartmental models, where the states represent counts of certain categories, the changes in these quantities

## Example: SIR model

Here we use a Susceptible-Infected-Removed epidemic model (an SIR model, for short) to demonstrate the different ways to specify a system.
The rates of change of each compartment are defined by the following equations

$$\begin{aligned}
\frac{\mathrm{d} S}{\mathrm{d} t} &= -\beta SI \\
\frac{\mathrm{d} I}{\mathrm{d} t} &= \beta SI- \gamma I \\
\frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I.
\end{aligned}$$

However, it is also true that this is the result of two transitions:

$$\begin{aligned}
S \rightarrow I &= \beta SI \\
I \rightarrow R &= \gamma I
\end{aligned}$$


We can set this up as follows:

1. Import the classes required to define the transitions

In [ ]:
from pygom import Transition, TransitionType

2. Define our states, in this case the population states of **S**usceptible, **I**nfected and **R**emoved

In [ ]:
stateList = ['S', 'I', 'R']

3. Define the set of parameters for our model

In [ ]:
paramList = ['beta', 'gamma']

4. 
  i. Specify the transitions of the modelled states; this will form our ODE system

In [ ]:
odeList = [
    Transition(origin='S', equation='-beta*S*I', transition_type=TransitionType.ODE),
    Transition(origin='I',equation='beta*S*I - gamma*I', transition_type=TransitionType.ODE),
    Transition(origin='R', equation='gamma*I', transition_type=TransitionType.ODE) 
]

ii. Specify the transitions of the modelled states; this will form our ODE system

In [ ]:
odeList2 = [
    Transition(origin='S', destination='I', equation='beta*S*I', transition_type=TransitionType.T),
    Transition(origin='I', destination='R', equation='gamma*I', transition_type=TransitionType.T)
]

```{note}
Here, we have invoked a class from {class}`.Transition` to define the ODE object. We proceed here and ignore the details for now. The details of defining a transition object will be covered later in {doc}`transition`. Both the set of states and parameters should be defined when constructing the object, even though not explicitly enforced, to help clarify what we are trying to model. Similarly, this holds for the rest, such as the derived parameters and transitions, where we force the end user to input the different type of transition/process via the correct argument. See {ref}`transition:defining-the-equations` for an example when the input is wrong.
```